# Data Preprocessing

Import libraries

In [1]:
import pandas as pd
import numpy as np


Loading the dataset

In [2]:
df = pd.read_csv(r'..\datasets\books_1.Best_Books_Ever.csv')
df.sample().T

,40778
bookId,490667.Sport
title,Sport: A Novel
series,NaN
author,Mick Cochrane
rating,4.02
description,A nostalgic story about a Minnesota boy's sear...
language,English
isbn,9780816640850
genres,[]
characters,[]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52478 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52478 non-null  object 
 1   title             52478 non-null  object 
 2   series            23470 non-null  object 
 3   author            52478 non-null  object 
 4   rating            52478 non-null  float64
 5   description       51140 non-null  object 
 6   language          48672 non-null  object 
 7   isbn              52478 non-null  object 
 8   genres            52478 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        51005 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             50131 non-null  object 
 13  publisher         48782 non-null  object 
 14  publishDate       51598 non-null  object 
 15  firstPublishDate  31152 non-null  object 
 16  awards            52478 non-null  object

Checking missing values

In [4]:
df.isna().sum()

bookId                  0
title                   0
series              29008
author                  0
rating                  0
description          1338
language             3806
isbn                    0
genres                  0
characters              0
bookFormat           1473
edition             47523
pages                2347
publisher            3696
publishDate           880
firstPublishDate    21326
awards                  0
numRatings              0
ratingsByStars          0
likedPercent          622
setting                 0
coverImg              605
bbeScore                0
bbeVotes                0
price               14365
dtype: int64

Dropping missing values from essential columns

In [5]:
df.dropna(subset=['coverImg', 'title', 'genres', 'author', 'description'], inplace=True)

Dropping duplicates (only when both `title` and `author` columns are the same)

In [6]:
df.drop_duplicates(subset=['title', 'author'], inplace=True)

Choosing only *English* books

In [7]:
df = df[df['language'] == 'English']

Removing insignificant descriptions

In [8]:
df = df[df['description'].str.len() > 75]

Saving preprocessed dataset

In [11]:
df.to_csv(r'..\datasets\preprocessed.csv', index=False)

Loading the preprocessed dataset

In [2]:
df = pd.read_csv(r'..\datasets\preprocessed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41617 entries, 0 to 41616
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            41617 non-null  object 
 1   title             41617 non-null  object 
 2   series            20420 non-null  object 
 3   author            41617 non-null  object 
 4   rating            41617 non-null  float64
 5   description       41617 non-null  object 
 6   language          41617 non-null  object 
 7   isbn              41617 non-null  object 
 8   genres            41617 non-null  object 
 9   characters        41617 non-null  object 
 10  bookFormat        41234 non-null  object 
 11  edition           3773 non-null   object 
 12  pages             40464 non-null  object 
 13  publisher         39307 non-null  object 
 14  publishDate       41298 non-null  object 
 15  firstPublishDate  25401 non-null  object 
 16  awards            41617 non-null  object

Saving dataframe to `MySQL` database

In [16]:
df['likedPercent'].fillna(value="missing", inplace=True)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_21216\3283585613.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['likedPercent'].fillna(value="missing", inplace=True)
C:\Users\Ruslan\AppData\Local\Temp\ipykernel_21216\3283585613.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'missing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['likedPercent'].fillna(value="missing", inplace=True)


In [ ]:
df = df.loc[:, ['title', 'author', 'genres', 'description', 'coverImg', 'likedPercent', 'rating',
                'numRatings', 'pages', 'bookFormat', 'isbn']]

In [ ]:
df['description'].str.len().describe()

count    41617.000000
mean       873.390778
std        531.101889
min         76.000000
25%        542.000000
50%        805.000000
75%       1091.000000
max      24733.000000
Name: description, dtype: float64

In [4]:
import mysql.connector
from dotenv import load_dotenv
import os


load_dotenv()

HOST = os.getenv("DB_HOST")
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")

mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="mydatabase"
)

In [5]:
cursor = mydb.cursor()

In [64]:
# create a "library" table in mysql if it does not exist
cursor.execute("DROP TABLE IF EXISTS library")
cursor.execute("""CREATE TABLE library (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title TEXT,
    author TEXT,
    genres TEXT,
    description TEXT,
    coverImg TEXT,
    likedPercent VARCHAR(50),
    rating FLOAT,
    numRatings INT,
    pages INT,
    bookFormat VARCHAR(50),
    isbn VARCHAR(50)
)""")

In [65]:
from sqlalchemy import create_engine

engine = create_engine(f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/mydatabase")

In [66]:
df.to_sql('library', con=engine, if_exists='append', index=False)

41617